<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo-T/blob/master/DataAlgo_T(004)_BFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.2. Breadth-First Search

We learn how to visit all the vertices in an undirected and connected graph by way of breadth-first search.

**Reminder**  
On github, rendering might not be in good shape.  
To see the expected layout, open this page in Google Colaboratory.
To run one specific code cell in colab, click the icon on the left part or just type Ctrl + Enter.  
  


# Preparation

Connect the Jupyter environment and invoke a runtime. 
Mount your Google Drive by the procedure below.  
Change directory to the mounted point and make it as the working folder.  
By then, files are preserved even after you terminate the runtime environment.

In [ ]:
!echo "Mounting your Google Drive"
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!echo "Make a working folder and chnage directory to it"
%cd /content/drive/My\ Drive
%mkdir -p DataAlgo-T/004
%cd       DataAlgo-T/004
!ls

Consider Graph G3.  
8 vertices, undirected, connected. 
It can be writen in C program by the form of adjacency matrix.
Element-1 means the edge exists betweeen row-ID-vertex and column-ID-vertex (they are adjacent).

![da2020-graph3](https://user-images.githubusercontent.com/45651568/84249701-1107b680-ab46-11ea-89b0-6a454ec80ec4.jpg)

In [ ]:
%%writefile graph3.h
// 8 vertices, undirected, no-weight.
#define N 8
int edge[N][N] = {
//   0 1 2 3 4 5 6 7
	{0,1,0,0,1,0,0,0}, // 0
	{1,0,0,0,0,0,1,0}, // 1
	{0,0,0,1,0,0,1,0}, // 2
	{0,0,1,0,0,0,0,1}, // 3
	{1,0,0,0,0,1,0,0}, // 4
 	{0,0,0,0,1,0,1,0}, // 5
	{0,1,1,0,0,1,0,1}, // 6
	{0,0,0,1,0,0,1,0}  // 7
};

# Outline of BFS

In dungeon-type RPG like Rogue, it is a breadth-first search to choose the room close to the first room (start vertex). It can be said that it is a careful strategy. 
 
More precisely, the search that enumerates the vertices close from the starting vertex is the width-first search. We assume all the edges in the graph shall have the same length. 
 
In the description of the algorithm of breadth-first search, a queue (First-In-First-Out) is used. We first investigate the vertex closest to the starting vertex and enumerate it. At this time, the unenumerated vertices adjacent to the vertex is put on the tail of the waiting line (connect them to the queue). When the investigation of a particular vertex is completed, the next vertex to investigate is the oldest one in the queue (First-In-First-Out). The reason for this is that it becomes the vertex farther from the starting vertex as the one connected later. 

For reference, when BFS was performed from vertex 0 for graph G3, the vertex enumeration by breadth-first is as follows.   
Here, if there are multiple vertices of the same priority, the lower ID number of vertices shall be selected first. 

BFS result: 0 1 4 6 5 2 7 3

![image](https://user-images.githubusercontent.com/45651568/90238137-6753e700-de60-11ea-9eac-0cd2ea206a7b.png)
Figure: 
BFS procedure (Graph G3, starting at 0)

# C program of BFS

**Purpose**

List up the vertices of a given graph in the order of breadth-first search that utilizes queue.

**Explanation**

A starting vertex is specified at the beginning.  

Queue could be a line in a supermarket.  
When an object newly arrived, it shall be placed at the tail of the queue. This is called enqueue operation.  
When the next turn comes, the object in the head of the queue is always processed. This is called dequeue operation.  
On dequeueing a vertex (at the head of the queue), the associateed adjacent vertices are enumalted and they are enqueued. 

The list up should always ends up with listing all the vertices in the graph if it is undirected and connected.

**Program**

The function bfs() is void because it never fails.
Since the data in the queue does not move, we need to prepare two variables to remember the head position and the tail position in the queue.

**Remarks**

On supermarket, people in the line walk as the procedure goes on. In such a case, the head position is fixed (in front of the register). But in the C program, data won't move. That is the reason why we need to remember head position too.

In [ ]:
%%writefile bfs_simple_E.c
// Breadth First Search.
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h> // printf()
#include "graph3.h"

// BFS main body
void bfs(int StartingRoom){
	int listed[N]; // 0 not yet, 1 queued
	int queue[N];
	int qhead = 0; // queue head position
	int qtail = 0; // queue tail position
    
    int CurrentRoom;
    int RoomToCheck;

	for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++)
		listed[RoomToCheck] = 0;
	
	printf("Start BFS from vertex %d.\n", StartingRoom);
    // Enqueue the starting vertex at the tail position of the queue
    queue[qtail] = StartingRoom;
    qtail++;

    // Mark as queued
	printf("Vertex %d is now on the queue.\n", StartingRoom);
    listed[StartingRoom] = 1;
 
    // Keep working if the vertices are in the queue.
	while (qhead < qtail) {
		printf("Queue  status: length %d, vertex at head is %d, vertex at tail is %d\n", qtail - qhead, queue[qhead], queue[qtail - 1]);
    	{int i; printf("Current queue: "); for (i = qhead; i < qtail; i++) printf("%d ", queue[i]); printf("\n");}

        // Invesitgate the vertex (taken from the head of the queue, removing it from the queue)
        printf("Take vertex %d on the queue head and start process (it is deleted from the queue).\n", queue[qhead]);
		CurrentRoom = queue[qhead];
        qhead++;

        // Some extra process may be placed here.

        // Choose next
		for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++) {
            if (edge[CurrentRoom][RoomToCheck] == 0) {
				printf("  (At Vertex %d) Vertex %d is not adjacent.\n", CurrentRoom, RoomToCheck);
            } else if (listed[RoomToCheck] != 0) {
				printf("  (At Vertex %d) Vertex %d is adjacent, but already listed before.\n", CurrentRoom, RoomToCheck);
            } else {         
				printf("  (At Vertex %d) Vertex %d is adjacent, and not listed, so put it on the tail of the queue.\n", CurrentRoom, RoomToCheck);
                // Enqueue RoomToCheck
				queue[qtail] = RoomToCheck;
                qtail++;

                // Once enqueued, it is recognized as being listed.
                printf("  (At Vertex %d) Vertex %d is treated as already listed from now.\n", CurrentRoom, RoomToCheck);
				listed[RoomToCheck] = 1;
			}
		}
	}
	printf("No vertices left in the queue. Quiting.\n");

	printf("Result of List up by BFS.\n");
	for (CurrentRoom = 0; CurrentRoom < N; CurrentRoom++)
		printf("%d ", queue[CurrentRoom]);
    printf("\n");

	return;
}

// Main function
int main(int argc, char *argv[]){
	bfs(0); // Call bfs() with the starting vertex ID 
	return 0;
}

Compile it and check no errors.

In [ ]:
!gcc -Wall -o bfs_simple_E bfs_simple_E.c

Run it.

In [ ]:
!./bfs_simple_E

# Coomand line option to set starting vertex

**Purpose**

Let the program take the command line option as the ID of the starting vertex to list up vertices by BFS.

**Explanation**

Use atoi() to convert string at command line option to integer.
(It is desirable to use strtol() since it can handle errors while atoi() cannot do it.)

**Program**

Command line option is processed within main().

**Remarks**

No changes at bfs() function.


In [ ]:
%%writefile bfs_xtraCommandLine_E.c
// Breadth First Search.
// kameda[at]ccs.tsukuba.ac.jp, 2020.
// -- Extra --
//    Accept command line option
#include <stdio.h> // printf()
#include <stdlib.h> // atoi()
#include "graph3.h"

// BFS main body
void bfs(int StartingRoom){
	int listed[N]; // 0 not yet, 1 queued
	int queue[N];
	int qhead = 0; // queue head position
	int qtail = 0; // queue tail position
    
    int CurrentRoom;
    int RoomToCheck;

	for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++)
		listed[RoomToCheck] = 0;
	
	printf("Start BFS from vertex %d.\n", StartingRoom);
    // Enqueue the starting vertex at the tail position of the queue
    queue[qtail] = StartingRoom;
    qtail++;

    // Mark as queued
	printf("Vertex %d is now on the queue.\n", StartingRoom);
    listed[StartingRoom] = 1;
 
    // Keep working if the vertices are in the queue.
	while (qhead < qtail) {
		printf("Queue  status: length %d, vertex at head is %d, vertex at tail is %d\n", qtail - qhead, queue[qhead], queue[qtail - 1]);
    	{int i; printf("Current queue: "); for (i = qhead; i < qtail; i++) printf("%d ", queue[i]); printf("\n");}

        // Invesitgate the vertex (taken from the head of the queue, removing it from the queue)
        printf("Take vertex %d on the queue head and start process (it is deleted from the queue.)\n", queue[qhead]);
		CurrentRoom = queue[qhead];
        qhead++;

        // Some extra process may be placed here.

        // Choose next
		for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++) {
            if (edge[CurrentRoom][RoomToCheck] == 0) {
				printf("  (At Vertex %d) Vertex %d is not adjacent.\n", CurrentRoom, RoomToCheck);
            } else if (listed[RoomToCheck] != 0) {
				printf("  (At Vertex %d) Vertex %d is adjacent, but already listed before.\n", CurrentRoom, RoomToCheck);
            } else {         
				printf("  (At Vertex %d) Vertex %d is adjacent, and not listed, so put it on the tail of the queue.\n", CurrentRoom, RoomToCheck);
                // Enqueue RoomToCheck
				queue[qtail] = RoomToCheck;
                qtail++;

                // Once enqueued, it is recognized as being listed.
                printf("  (At Vertex %d) Vertex %d is treated as already listed from now.\n", CurrentRoom, RoomToCheck);
				listed[RoomToCheck] = 1;
			}
		}
	}
	printf("No vertices left in the queue. Quiting.\n");

	printf("Result of List up by BFS.\n");
	for (CurrentRoom = 0; CurrentRoom < N; CurrentRoom++)
		printf("%d ", queue[CurrentRoom]);
    printf("\n");

	return;
}

// Main function
int main(int argc, char *argv[]){
    int startvertex = 0;
    if (argc == 1) {
        printf("Start vertex not specified, so set No.0 as start.\n");
        startvertex = 0;
    } else if (argc == 2) {
        startvertex = atoi(argv[1]);
        if (startvertex < 0 || startvertex >= N) {
            printf("Illegal ID specified, so quit.\n");
            return -1;
        }
    }

	bfs(startvertex);  // Call bfs() with the starting vertex ID 
	return 0;
}

Compile it and check no errors.

In [ ]:
!gcc -Wall -o bfs_xtraCommandLine_E bfs_xtraCommandLine_E.c

Run it.

In [ ]:
!./bfs_xtraCommandLine_E 2

# C program of BFS by function call

**Purpose**

Enqueue and dequeue by user defined functions.

**Explanation**

Write user defined functions in a different source file.  
It could be treated as an external library in the future.．

**Program**

Original source file is splitted into three parts.
* a header file of queue operation
* source file of user defined functions
* application source file of BFS

**Remarks**

Separate compilation method is applied.

In [ ]:
%%writefile QueueLib_E.h
// Queue management
// kameda[at]ccs.tsukuba.ac.jp, 2020.

// Initialization of queue
int *initqueue(int );

// Add to the queue
int enqueue(int );

// Delete from the queue
int dequeue(void);

// Show the status of the queue
int showqueue(void);

// Show the whole status of the queue
int showallqueue(void);


In [ ]:
%%writefile QueueLib_E.c
// Queue management
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h> // printf()
#include <stdlib.h> // calloc()
#include "QueueLib_E.h" // Prototype check

int *queue = NULL; // Queue mainbody (needs initialization)
int qsize = 0; // Queue size
int qhead = 0; // Queue head position in queue[]
int qtail = 0; // Queue tail position in queue[]

// Initialization of queue
int *initqueue(int n) {
    queue = calloc(n, sizeof(*queue));
    qsize = n;
    if (queue == NULL) {
        printf("[Error] Cannot obtain queue memory for %d elements.\n", n);
        qsize = 0;
    }
    return queue;
}

// Add to the queue
int enqueue(int v) {
    if (queue == NULL) {
        printf("[Error] Queue is not ready.\n");
        return -1;
    } else if (qtail >= qsize) {
        printf("[Error] Running out of queue memory. Currently %d elements.\n", qsize);
        return -2;
    }
    queue[qtail] = v;
    qtail++;
    return 0;
}

// Delete from the queue
int dequeue(void) {
    int v;
    if (queue == NULL) {
        printf("[Error] Queue is not ready.\n");
        return -1;
    } else if (qhead >= qtail) {
        printf("[Report] No elements in the queue.\n");
        return -2;
    }
    v = queue[qhead];
    qhead++;
    return v;
}

// Show the status of the queue
int showqueue(void) {
    int i;

    if (queue == NULL) {
        printf("[Error] Queue is not ready.\n");
        return -1;
    }

    printf("qsize = %d, qhead = %d, qtail = %d\n", qsize, qhead, qtail);
    for (i = qhead; i < qtail; i++) {
        printf("%d ", queue[i]);
    }
    printf("\n");

    return qtail - qhead;
}

// Show the whole status of the queue
int showallqueue(void) {
    int i;

    if (queue == NULL) {
        printf("[Error] Queue is not ready.\n");
        return -1;
    }

    printf("qsize = %d, qhead = %d, qtail = %d\n", qsize, qhead, qtail);
    for (i = 0; i < qsize; i++) {
        printf("%d ", queue[i]);
    }
    printf("\n");

    return qsize;
}



Compile only the source file of user defined functions.
This is equivalent to create a new (user degined) library.

In [ ]:
!gcc -Wall -c QueueLib_E.c

Rewrite the bfs() function to use external user-defined functions for handling the queue.  
QueueLib_E.h will be a help as a quick reference.

In [ ]:
%%writefile bfs_xtraFuncCalls_E.c
// Breadth First Search.
// kameda[at]ccs.tsukuba.ac.jp, 2020.
// -- Extra --
//    Accept command line option
//    Call functions for enqueue and dequeue
#include <stdio.h> // printf()
#include <stdlib.h> // atoi()
#include "QueueLib_E.h"
#include "graph3.h"

// BFS main body
void bfs(int StartingRoom){
    int listed[N]; // 0 not yet, 1 queued
    
    int CurrentRoom;
    int RoomToCheck;

	for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++)
		listed[RoomToCheck] = 0;

    // Initialization of queue
    initqueue(N);
	
	printf("Start BFS at Vertex %d.\n", StartingRoom);
    // Enqueue StartingRoom
    enqueue(StartingRoom);

    // Once enqueued, it is recognized as being listed.
    printf("Vertex %d is treated as already listed from now.\n", RoomToCheck);
    listed[StartingRoom] = 1;

    // Show the status of the queue
    // the timing to show the status has been moved to this position so as to show the status just before enqueue that is done in while() eveluation
    showqueue();

    // Keep working if the vertices are in the queue
	while ((CurrentRoom = dequeue()) >= 0) {
        // Invesitgate the vertex (taken from the head of the queue, removing it from the queue)
		printf("Take vertex %d on the queue head and start process (it is deleted from the queue.)\n", CurrentRoom);
        
        // Some extra process may be placed here

        // Choose next
		for (RoomToCheck = 0; RoomToCheck < N; RoomToCheck++) {
            if (edge[CurrentRoom][RoomToCheck] == 0) {
				printf("  (At Vertex %d) Vertex %d is not adjacent.\n", CurrentRoom, RoomToCheck);
            } else if (listed[RoomToCheck] != 0) {
				printf("  (At Vertex %d) Vertex %d is adjacent, but already listed before.\n", CurrentRoom, RoomToCheck);
            } else {         
				printf("  (At Vertex %d) Vertex %d is adjacent, and not listed, so put it on the tail of the queue.\n", CurrentRoom, RoomToCheck);
                // Enqueue RoomToCheck
				enqueue(RoomToCheck);
                
                // Once enqueued, it is recognized as being listed.
                printf("  (At Vertex %d) Vertex %d is treated as already listed from now.\n", CurrentRoom, RoomToCheck);
				listed[RoomToCheck] = 1;
			}
		}
        // Show the status of the queue
        // the timing to show the status has been moved to this position so as to show the status just before enqueue that is done in while() eveluation
        showqueue();
	}
	printf("No vertices left in the queue. Quiting.\n");

	printf("Result of List up by BFS.\n");
    showallqueue();

	return;
}

// Main function
int main(int argc, char *argv[]){
    int startvertex = 0;
    if (argc == 1) {
        printf("Start vertex not specified, so set No.0 as start.\n");
        startvertex = 0;
    } else if (argc == 2) {
        startvertex = atoi(argv[1]);
        if (startvertex < 0 || startvertex >= N) {
            printf("Illegal ID specified, so quit.\n");
            return -1;
        }
    }

	bfs(startvertex);  // Call bfs() with the starting vertex ID 
	return 0;
}

Compile the bfs program part.

In [ ]:
!gcc -Wall -c bfs_xtraFuncCalls_E.c

Link the object files (*.o) to make the binary file (bfs_xtraFuncCalls_E).

In [ ]:
!gcc -o bfs_xtraFuncCalls_E bfs_xtraFuncCalls_E.o QueueLib_E.o

Run it. 
Output should be same as the previous one.

In [ ]:
!./bfs_xtraFuncCalls_E 0

# Problems

1. Description of the algorithm   
Describe the breadth-first search algorithm using queues in Human’s language.   
The description must follow be the explanation in  this lecture.    
The diagram of the queue should be inserted. 

2. Complete algorithm   
As long as a given graph is undirected and connected, the breadth-first search algorithm shown in this section always list up all vertices successfully. Try to explain that this is true. 


3. Computation amount   
Discuss the computation amount of time and space of the program in bfs_xtraCommandLine_E.c. 

4. Memory size of queue[]   
No matter what undirected and connected graph is specified, show that the memory size required for queue[] does not exceed the number of vertices. 

5. Longest queue   
Find out the shape of a graph that gives the longest queue tentatively on processing the graph. (In the program, the queue length is qtail – qhead + 1.) 

6. Show distance from the starting vertex   
Modifiy the program (bfs_xtraCommandLine_E.c) to show the distance of each vertex from the starting vertex in numbers on showing the result of the list up. The distance is indicated by the number of edges from the starting vertex.   
(e.g.) BFS result: 0(0) 1(1) 4(1) 6(2) 5(2) 2(3) 7(3) 3(4) 

7. About the atoi() function   
(a) Describe what happens if you pass a string that cannot be interpreted as an integer to the atoi() function.  
(b) Show what actually happens if you pass a string that cannot be interpreted as an integer to the atoi() function.  

8. About strtol() functions   
(a) Describe how the strtol() function traps errors.  
(b) Write a simple program that traps error with strtol() function.   
(Tip: Stortol() function uses errno to handle errors.) 

9. Moving queue   
Modify the program (bfs_xtraCommandLine_E.c) that move the elements of queue[] forward when dequeueing a vertex from queue[]. (In other words, make sure that queue[0] is always at the head position of the queue like people do at supermarket.) 

10. Specifying a graph   
Instead of including "graph3.h", let the program (bfs_xtraCommandLine_E.c) accept a text file describing the adjacent matrix of the graph. The text filename should be given by a command line option.  






#**Course Info**

https://github.com/kameda-yoshinari/DataAlgo-T  
Course: Data structure and algorithm  
Department of Engineering Systems, University of Tsukuba,Japan.  
Author: KAMEDA, Yoshinari  
2020.05.19. -  